In [2]:
import utils
import datetime
import pandas as pd 
import instrument
import plotly.graph_objects as go

def get_price_data(pairname, granularity):
    df = pd.read_pickle(utils.get_his_data_filename(pairname, granularity))
    non_cols = ["time", "volume"]
    mod_cols = [x for x in df.columns if x not in non_cols]
    df[mod_cols] = df[mod_cols].apply(pd.to_numeric)
    return df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]]

def get_test_pairs(pair_str):
    existing_pairs = instrument.Instrument.get_instruments_dict().keys()
    pairs = pair_str.split(",")
    test_list = []
    for p1 in pairs:
        for p2 in pairs:
            p = f"{p1}_{p2}"
            if p in existing_pairs:
                test_list.append(p)
    return test_list

In [27]:
def create_candle_plot(df_plot, range=None, start=None):
    fig = go.Figure()
    
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot["MA_4"],
        line=dict(width=2),
        line_shape='spline',
    ))
    if range is not None:
        fig.add_hline(y=range[0], line_width=2, line_dash="dash", line_color="yellow")
        fig.add_hline(y=range[1], line_width=2, line_dash="dash", line_color="yellow")
        
    if start is not None:
        fig.add_vline(x=start, line_width=2, line_dash="dash", line_color="blue")
    
    fig.update_layout(width=1000,height=300,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [37]:
currencies = "GBP,EUR,USD,CAD,JPY,NZD,CHF"
granularity = "M5"
test_pairs = get_test_pairs(currencies)
test_pairs

['GBP_USD',
 'GBP_CAD',
 'GBP_JPY',
 'GBP_NZD',
 'GBP_CHF',
 'EUR_GBP',
 'EUR_USD',
 'EUR_CAD',
 'EUR_JPY',
 'EUR_NZD',
 'EUR_CHF',
 'USD_CAD',
 'USD_JPY',
 'USD_CHF',
 'CAD_JPY',
 'CAD_CHF',
 'NZD_USD',
 'NZD_CAD',
 'NZD_JPY',
 'NZD_CHF',
 'CHF_JPY']

In [47]:
for pairname in test_pairs:
    i_pair = instrument.Instrument.get_instruments_dict()[pairname]
    print(i_pair)

{'name': 'GBP_USD', 'ins_type': 'CURRENCY', 'displayName': 'GBP/USD', 'pipLocation': 0.0001, 'marginRate': '0.03333333333333'}
{'name': 'GBP_CAD', 'ins_type': 'CURRENCY', 'displayName': 'GBP/CAD', 'pipLocation': 0.0001, 'marginRate': '0.03333333333333'}
{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'marginRate': '0.03333333333333'}
{'name': 'GBP_NZD', 'ins_type': 'CURRENCY', 'displayName': 'GBP/NZD', 'pipLocation': 0.0001, 'marginRate': '0.05'}
{'name': 'GBP_CHF', 'ins_type': 'CURRENCY', 'displayName': 'GBP/CHF', 'pipLocation': 0.0001, 'marginRate': '0.04'}
{'name': 'EUR_GBP', 'ins_type': 'CURRENCY', 'displayName': 'EUR/GBP', 'pipLocation': 0.0001, 'marginRate': '0.03333333333333'}
{'name': 'EUR_USD', 'ins_type': 'CURRENCY', 'displayName': 'EUR/USD', 'pipLocation': 0.0001, 'marginRate': '0.03333333333333'}
{'name': 'EUR_CAD', 'ins_type': 'CURRENCY', 'displayName': 'EUR/CAD', 'pipLocation': 0.0001, 'marginRate': '0.03333333333333'}
{'name': '

In [48]:
pairname = 'GBP_NZD'
price_data = get_price_data(pairname, granularity)
i_pair = instrument.Instrument.get_instruments_dict()[pairname]

In [52]:
# Trading Day
day = '2018-02-01'

# Start and end time for trading day
start_time = utils.get_utc_dt_from_string(f'{day} 00:00:00+00:00')
end_time =  start_time + datetime.timedelta(days=1)

# Extract price data for trading day
df_day = price_data[(price_data.time >= start_time) & (price_data.time < end_time) ].copy()
df_day.dropna(inplace=True)
df_day.reset_index(drop=True, inplace=True)

# Extract pre-open period
asia_start_time = utils.get_utc_dt_from_string(f'{day} 00:00:00+00:00')
asia_end_time = asia_start_time + datetime.timedelta(hours=9)
london_start_time = utils.get_utc_dt_from_string(f'{day} 07:00:00+00:00')
london_end_time = london_start_time + datetime.timedelta(hours=9)

df_pre_london_open = df_day[(df_day.time >= asia_start_time) & (df_day.time < london_start_time)].copy()
df_post_london_open = df_day[(df_day.time >= london_start_time)].copy()

# Find opening Asian trading range
range_max = max(df_pre_london_open.mid_c.max(), df_pre_london_open.mid_o.max())
range_min = min(df_pre_london_open.mid_c.min(), df_pre_london_open.mid_o.min())

print(range_min, range_max)

print((range_max - range_min)/i_pair.pipLocation)

df_day[f'MA_4'] = df_day.mid_c.rolling(window=4).mean()

1.9249 1.93063
57.300000000000125


In [53]:
create_candle_plot(df_day, range=(range_max, range_min), start=london_start_time)

In [54]:
BUY = 1
SELL = -1

def get_close_time_as_string(df):
    close_time = df.loc[(len(df.index) - 1), "time"].strftime("%H:%M:%S")
    return close_time

def get_past_data(df, index):
    return df.iloc[:index].copy()

def count_open_trades(df):
    buy_list = [x for x in df.IS_TRADE if x == BUY]
    sell_list = [x for x in df.IS_TRADE if x == SELL]
    buys = len(buy_list)
    sells = len(sell_list)
    return {"total": (buys + sells), "buys": buys, "sells": sells}

def over_range_max(row, range_max):
    return row.mid_o >= range_max and row.mid_c > range_max

def under_range_min(row, range_min):
    return row.mid_o <= range_min and row.mid_c < range_min

def inside_range(row, range_min, range_max):
    check_open = row.mid_o > range_min and row.mid_o < range_max
    check_close = row.mid_c > range_min and row.mid_c < range_max
    return check_open and check_close

df = df_post_london_open.copy()
df.reset_index(drop=True, inplace=True)

df['IS_TRADE'] = 0
for index, row in df.iterrows():
    trade_count = count_open_trades(get_past_data(df, index))
    if over_range_max(row, range_max):
        if trade_count['buys'] == 0:
            df.loc[index, 'IS_TRADE'] = BUY
    elif under_range_min(row, range_min):
        if trade_count['sells'] == 0:
            df.loc[index, 'IS_TRADE'] = SELL
    elif inside_range(row, range_min, range_max):
        if trade_count['buys'] == 1 and trade_count['total'] == 1:
            df.loc[index, 'IS_TRADE'] = SELL
        if trade_count['sells'] == 1 and trade_count['total'] == 1:
            df.loc[index, 'IS_TRADE'] = BUY
    if row.time >= utils.get_utc_dt_from_string(f'{day} {get_close_time_as_string(df)}'):
        print(row.time.strftime("%H:%M:%S"))
        if trade_count["total"] == 1:
            if trade_count["buys"] ==1:
                df.loc[index, 'IS_TRADE'] = SELL
            if trade_count["sells"] ==1:
                df.loc[index, 'IS_TRADE'] = BUY

df_trades = df[df.IS_TRADE != 0 ].copy()
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]*(-1)
df_trades.head(100)


23:55:00


,time,mid_o,mid_h,mid_l,mid_c,IS_TRADE,DELTA,GAIN
11,2018-02-01 07:55:00+00:00,1.93135,1.93374,1.93135,1.93336,1,NaN,NaN
87,2018-02-01 14:15:00+00:00,1.92971,1.93004,1.92932,1.92936,-1,-40.0,-40.0


In [192]:
week_1= 10*(59) + 10*(75.6) + 10*(23.9) + 10*(-3.6)
week_2 = 10*(-4.5) + 10*(21.5) + 10*(-7.9) + 10*(-8.9) +10*(154)
week_3 = 10*(-15.2) + 10*(-9.4) + 10*(-20.2) + 10*(34.9) + 10*(-13.6)
week_4 = 10*(-9.1) + 10*(-20.7) + 10*(170.2) + 10*(52.0) + 10*(-42.9)
week_5 = 10*(-18.4) + 10*(-28.2) + 10*(-10) + 10*(-14.5) + 10*(130.4)

print('Week 1:', week_1)
print('Week 2:', week_2)
print('Week 3:', week_3)
print('Week 4:', week_4)
print('Week 5:', week_5)

total = week_1 + week_2 + week_3 + week_4 + week_5

print('Total:', total)

Week 1: 1549.0
Week 2: 1542.0
Week 3: -235.0
Week 4: 1495.0
Week 5: 593.0
Total: 4944.0


In [196]:
# Find point after London open when asian trading range is broken
df_day['mid_c_next'] = df_day.mid_c.shift(-1)
df_day['cross_max'] = (
    (((df_day.mid_c >= range_max) & (df_day.mid_c_next < range_max)) |
    ((df_day.mid_c_next >= range_max) & (df_day.mid_c < range_max)))
    & (df_day.time >= (london_start_time - datetime.timedelta(hours=0.25)))
)
df_day['cross_min'] = (
    (((df_day.mid_c >= range_min) & (df_day.mid_c_next < range_min)) |
    ((df_day.mid_c_next >= range_min) & (df_day.mid_c < range_min)))
    & (df_day.time >= (london_start_time - datetime.timedelta(hours=0.25)))
)

df_day.head(100)

,time,mid_o,mid_h,mid_l,mid_c,mid_c_next,cross_max,cross_min,MA_8
312,2018-01-03 00:00:00+00:00,1.35946,1.35970,1.35942,1.35968,1.35952,False,False,NaN
313,2018-01-03 00:05:00+00:00,1.35969,1.35994,1.35942,1.35952,1.35966,False,False,NaN
314,2018-01-03 00:10:00+00:00,1.35952,1.35966,1.35952,1.35966,1.35964,False,False,NaN
315,2018-01-03 00:15:00+00:00,1.35958,1.35971,1.35954,1.35964,1.35970,False,False,NaN
316,2018-01-03 00:20:00+00:00,1.35957,1.35978,1.35956,1.35970,1.35952,False,False,NaN
...,...,...,...,...,...,...,...,...,...
407,2018-01-03 08:00:00+00:00,1.35950,1.36020,1.35938,1.36007,1.35922,False,False,1.359408
408,2018-01-03 08:05:00+00:00,1.36006,1.36016,1.35912,1.35922,1.35899,False,True,1.359387
409,2018-01-03 08:10:00+00:00,1.35922,1.35938,1.35884,1.35899,1.35894,False,False,1.359362
410,2018-01-03 08:15:00+00:00,1.35906,1.35969,1.35894,1.35894,1.35962,False,True,1.359317


In [10]:
# Find first candle fully above or below the range lines
df_day.head(500)

,time,mid_o,mid_h,mid_l,mid_c
0,2018-01-18 00:00:00+00:00,1.38246,1.38354,1.38244,1.38332
1,2018-01-18 00:05:00+00:00,1.38339,1.38396,1.38317,1.38370
2,2018-01-18 00:10:00+00:00,1.38370,1.38379,1.38340,1.38370
3,2018-01-18 00:15:00+00:00,1.38372,1.38376,1.38320,1.38320
4,2018-01-18 00:20:00+00:00,1.38320,1.38321,1.38294,1.38300
...,...,...,...,...,...
283,2018-01-18 23:35:00+00:00,1.38950,1.38971,1.38936,1.38944
284,2018-01-18 23:40:00+00:00,1.38944,1.38946,1.38929,1.38929
285,2018-01-18 23:45:00+00:00,1.38930,1.38944,1.38921,1.38930
286,2018-01-18 23:50:00+00:00,1.38922,1.38940,1.38881,1.38916


In [60]:
def is_trade(row):
    if row.CROSS_MAX == True: 
        # Buy
        return 1
    if row.CROSS_MIN == True:
        # Sell 
        return -1
    return 0

# Find candles fully over or under the thresholds
df_day['OVER_MAX'] = ((df_day.mid_o >= range_max) & (df_day.mid_c > range_max))
df_day['OVER_MAX_PREV'] = df_day.OVER_MAX.shift(1)
df_day['UNDER_MIN'] = ((df_day.mid_o <= range_min) & (df_day.mid_c < range_min))
df_day['UNDER_MIN_PREV'] = df_day.UNDER_MIN.shift(1)

# Shift and compare to find first True instance
df_day['CROSS_MAX'] = ((df_day.OVER_MAX== True) & (df_day.OVER_MAX_PREV == False))
df_day['CROSS_MIN'] = ((df_day.UNDER_MIN == True) & (df_day.UNDER_MIN_PREV == False))

# Convert cross points into buy/sell signals
df_day["IS_TRADE"] = df_day.apply(is_trade, axis=1)

# Remove unused columns
del df_day['OVER_MAX']
del df_day['OVER_MAX_PREV']
del df_day['UNDER_MIN']
del df_day['UNDER_MIN_PREV']
del df_day['CROSS_MAX']
del df_day['CROSS_MIN']




In [61]:
df_trades = df_day[(df_day.IS_TRADE == 1) | (df_day.IS_TRADE == -1) ].copy()
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,IS_TRADE
130,2018-01-18 10:50:00+00:00,1.3849,1.38576,1.38474,1.38575,1


In [13]:
df_day.head()

,time,mid_o,mid_h,mid_l,mid_c,IS_TRADE
0,2018-01-18 00:00:00+00:00,1.38246,1.38354,1.38244,1.38332,0
1,2018-01-18 00:05:00+00:00,1.38339,1.38396,1.38317,1.38370,0
2,2018-01-18 00:10:00+00:00,1.38370,1.38379,1.38340,1.38370,0
3,2018-01-18 00:15:00+00:00,1.38372,1.38376,1.38320,1.38320,0
4,2018-01-18 00:20:00+00:00,1.38320,1.38321,1.38294,1.38300,0


In [281]:
# Loop through days between start and end dates
start_date = utils.get_utc_dt_from_string("2018-01-01")
end_date = utils.get_utc_dt_from_string("2020-12-31")
day_count = (end_date - start_date).days + 1
for day in (start_date + datetime.timedelta(n) for n in range(day_count)):
    day = day.strftime("%Y-%m-%d")

2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-13
2018-01-14
2018-01-15
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-20
2018-01-21
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-27
2018-01-28
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-03
2018-02-04
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-10
2018-02-11
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-17
2018-02-18
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-24
2018-02-25
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-03
2018-03-04
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-10
2018-03-11
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-17
2018-03-18
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-24
2018-03-25
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-03-30
2018-03-31
2018-04-01

In [82]:
trade_list = [x for x in df_day.IS_TRADE if x != 0]

trade_list = [1, -1, 1, -1]

print(trade_list[-1:].pop())

trade_list

-1


[1, -1, 1, -1]

In [14]:
i=df_day.index[df_day['time'] == utils.get_utc_dt_from_string(f'{day} 23:55:00+00:00')].tolist()
df_day.loc[i.pop(), 'IS_TRADE'] = -1

In [15]:
df_day.tail()

,time,mid_o,mid_h,mid_l,mid_c,IS_TRADE
283,2018-01-18 23:35:00+00:00,1.38950,1.38971,1.38936,1.38944,0
284,2018-01-18 23:40:00+00:00,1.38944,1.38946,1.38929,1.38929,0
285,2018-01-18 23:45:00+00:00,1.38930,1.38944,1.38921,1.38930,0
286,2018-01-18 23:50:00+00:00,1.38922,1.38940,1.38881,1.38916,0
287,2018-01-18 23:55:00+00:00,1.38908,1.38932,1.38884,1.38924,-1


In [16]:
df_day.IS_TRADE.sum()

0

In [17]:
df_trades = df_day[df_day.IS_TRADE != 0].copy()
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]*(-1)
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,IS_TRADE,DELTA,GAIN
130,2018-01-18 10:50:00+00:00,1.38490,1.38576,1.38474,1.38575,1,NaN,NaN
287,2018-01-18 23:55:00+00:00,1.38908,1.38932,1.38884,1.38924,-1,34.9,34.9


In [89]:
i_pair.pipLocation

0.0001

In [102]:
(1.37322 - 1.35778)/0.0001

154.4000000000012

In [38]:
date = df_day.loc[0, 'time'].strftime("%Y-%m-%d")


'2018-01-18'

numpy.datetime64('2018-01-18T00:00:00.000000000')

In [39]:
len(df_day.index)

288

In [46]:
date = df_day.loc[(len(df_day.index) - 1), 'time'].strftime("%Y-%m-%d")
date

'2018-01-18'

In [56]:
close_time = df_day.loc[(len(df_day.index) - 1), 'time'].strftime("%H:%M:%S")
print(close_time)

23:55:00


2018-01-18 00:00:00+00:00
